In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import Select
import json
import time


In [1]:
def wait(seconds):
    time.sleep(seconds)

def append_to_json_file(data, file_path="results.json"):
    try:
        with open(file_path, "r") as json_file:
            existing_data = json.load(json_file)
    except (FileNotFoundError, json.JSONDecodeError):
        existing_data = []

    existing_data.append(data)

    with open(file_path, "w") as json_file:
        json.dump(existing_data, json_file, indent=4)


In [2]:

def fill_field(driver, field_id, value):
    field = driver.find_element(By.ID, field_id)
    field.send_keys(value)

In [3]:
def select_dropdown_by_visible_text(driver, dropdown_id, text):
    dropdown = driver.find_element(By.ID, dropdown_id)
    dropdown.click()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f"//select[@id='{dropdown_id}']/option[text()='{text}']")))
    option_xpath = f"//select[@id='{dropdown_id}']/option[text()='{text}']"
    option = driver.find_element(By.XPATH, option_xpath)
    option.click()

def wait_for_dropdown_options(driver, dropdown_id):
    wait(2)

def upload_file(driver, file_input_id, file_path):
    file_input = driver.find_element(By.ID, file_input_id)
    file_input.send_keys(file_path)

def submit_form(driver):
    register_button = driver.find_element(By.ID, "register_submit")
    register_button.click()

def check_for_error(driver, error_id):
    try:
        error_element = driver.find_element(By.ID, error_id)
        return error_element.text if error_element.is_displayed() else ""
    except NoSuchElementException:
        return ""

In [4]:
def fill_input_fields(driver, email, cnic, company_code, mobile, password, confirm_password, country, state, city, source, file_path):
    fill_field(driver, "rec_reg_email", email)
    fill_field(driver, "rec_reg_cnic", cnic)
    fill_field(driver, "company_code", company_code)
    fill_field(driver, "rec_reg_contact_no", mobile)
    fill_field(driver, "rec_reg_password", password)
    fill_field(driver, "rec_reg_confirm_passoword", confirm_password)
    wait(5)
    select_dropdown_by_visible_text(driver, "country_id", country)
    wait_for_dropdown_options(driver, "state_id")
    select_dropdown_by_visible_text(driver, "state_id", state)
    wait_for_dropdown_options(driver, "city_id")
    select_dropdown_by_visible_text(driver, "city_id", city)
    select_dropdown_by_visible_text(driver, "rec_source_id", source)
    upload_file(driver, "file", file_path)

In [5]:
def fill_form(email, cnic, company_code, mobile, password, confirm_password, country, state, city, source, file_path):
    driver = webdriver.Chrome()
    error_text = ""

    try:
        driver.get("https://rec-demo-client.botnosticsolutions.ai/")
        fill_input_fields(driver, email, cnic, company_code, mobile, password, confirm_password, country, state, city, source, file_path)

        error_text = check_for_error(driver, "error-email")
        if not error_text:
            error_text = check_for_error(driver, "error-pwd-format")
        if not error_text:
            error_text = check_for_error(driver, "error-pwd")

        data = {
            "email": email,
            "cnic": cnic,
            "company_code": company_code,
            "mobile": mobile,
            "password": password,
            "confirm_password": confirm_password,
            "country": country,
            "state": state,
            "city": city,
            "source": source,
            "file_path": file_path,
            "error_text": error_text
        }

        append_to_json_file(data)

    except Exception as e:
        print("Error occurred during form filling:", e)
        import traceback
        traceback.print_exc()

        data = {
            "email": email,
            "cnic": cnic,
            "company_code": company_code,
            "mobile": mobile,
            "password": password,
            "confirm_password": confirm_password,
            "country": country,
            "state": state,
            "city": city,
            "source": source,
            "file_path": file_path,
            "error_text": error_text or "Exception occurred"
        }

        append_to_json_file(data)

    finally:
        wait(5)
        driver.quit()

In [6]:
# Test Cases
print("Running test cases...")

Running test cases...


In [9]:
# Valid form submission
print("Test Case: Valid form submission")
fill_form("valid@example.com", "1234567890123", "DC001", "03001234567", "ValidPass123!", "ValidPass123!", "Pakistan", "Punjab", "Lahore", "Facebook", "C://Users//MHS//Downloads//Media Assessment.pdf")


Test Case: Valid form submission


In [11]:
# Email format error
print("Test Case: Invalid email format")
fill_form("invalidemail.com", "1234567890123", "DC001", "03001234567", "ValidPass123!", "ValidPass123!", "Pakistan", "Punjab", "Lahore", "Facebook", "C://Users//MHS//Downloads//Media Assessment.pdf")


Test Case: Invalid email format


In [12]:
# Example usage: Password mismatch
print("Test Case: Password mismatch")
fill_form(
    "mismatch@example.com",
    "1234567890123",
    "DC001",
    "03001234567",
    "ValidPass123!",
    "MismatchPass123!",
    "Pakistan",
    "Punjab",
    "Lahore",
    "Facebook",
    "C://Users//MHS//Downloads//Media Assessment.pdf"
)

Test Case: Password mismatch


In [13]:
# Missing CNIC
print("Test Case: Missing CNIC")
fill_form(
    "missingcnic@example.com", 
    "", 
    "DC001", 
    "03001234567", 
    "ValidPass123!", 
    "ValidPass123!", 
    "Pakistan", 
    "Punjab", 
    "Lahore", 
    "Facebook", 
    "C://Users//MHS//Downloads//Media Assessment.pdf"
)
    

Test Case: Missing CNIC


In [14]:
# Invalid password format
print("Test Case: Invalid password format")
fill_form(
    "invalidpassword@example.com", 
    "1234567890123", 
    "DC001", 
    "03001234567", 
    "short", 
    "short", 
    "Pakistan", 
    "Punjab", 
    "Lahore", 
    "Facebook", 
    "C://Users//MHS//Downloads//Media Assessment.pdf"
)

Test Case: Invalid password format


In [15]:
# Non-existent country/state/city
print("Test Case: Non-existent country/state/city")
fill_form(
    "nonexistent@example.com", 
    "1234567890123", 
    "DC001", 
    "03001234567", 
    "ValidPass123!", 
    "ValidPass123!", 
    "NonExistentCountry", 
    "NonExistentState", 
    "NonExistentCity", 
    "Facebook", 
    "C://Users//MHS//Downloads//Media Assessment.pdf"
)

Test Case: Non-existent country/state/city
Error occurred during form filling: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF659171F52+60322]
	(No symbol) [0x00007FF6590ECEC9]
	(No symbol) [0x00007FF658FA7EBA]
	(No symbol) [0x00007FF658FF7676]
	(No symbol) [0x00007FF658FF773C]
	(No symbol) [0x00007FF65903E967]
	(No symbol) [0x00007FF65901C25F]
	(No symbol) [0x00007FF65903BC80]
	(No symbol) [0x00007FF65901BFC3]
	(No symbol) [0x00007FF658FE9617]
	(No symbol) [0x00007FF658FEA211]
	GetHandleVerifier [0x00007FF6594894AD+3301629]
	GetHandleVerifier [0x00007FF6594D36D3+3605283]
	GetHandleVerifier [0x00007FF6594C9450+3563680]
	GetHandleVerifier [0x00007FF659224326+790390]
	(No symbol) [0x00007FF6590F750F]
	(No symbol) [0x00007FF6590F3404]
	(No symbol) [0x00007FF6590F3592]
	(No symbol) [0x00007FF6590E2F9F]
	BaseThreadInitThunk [0x00007FFA08967344+20]
	RtlUserThreadStart [0x00007FFA095C26B1+33]



Traceback (most recent call last):
  File "C:\Users\MHS\AppData\Local\Temp\ipykernel_14556\1133378030.py", line 7, in fill_form
    fill_input_fields(driver, email, cnic, company_code, mobile, password, confirm_password, country, state, city, source, file_path)
  File "C:\Users\MHS\AppData\Local\Temp\ipykernel_14556\3674994775.py", line 9, in fill_input_fields
    select_dropdown_by_visible_text(driver, "country_id", country)
  File "C:\Users\MHS\AppData\Local\Temp\ipykernel_14556\2044416111.py", line 4, in select_dropdown_by_visible_text
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f"//select[@id='{dropdown_id}']/option[text()='{text}']")))
  File "C:\Users\MHS\anaconda3\lib\site-packages\selenium\webdriver\support\wait.py", line 105, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF659171F52+60322]
	(No symbol) [0x00007FF6590ECEC9]
	(No symb